In [1]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM

from pandas import read_csv, concat

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score

import tldextract
import numpy as np

2024-02-12 19:38:09.848301: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-12 19:38:09.848439: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-12 19:38:10.094346: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-12 19:38:10.641019: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-12 19:38:13.741922: W tensorflow/compiler/tf2

In [2]:
legitDomains = read_csv('..//data/top-1m.csv', names=['domain'])
dgaDomains = read_csv('../data/dga_domains.in.txt', names=['domain'])

legitDomains['tld'] = [tldextract.extract(d).domain for d in legitDomains['domain']]
dgaDomains['tld'] = [tldextract.extract(d).domain for d in dgaDomains['domain']]

legitDomains = legitDomains[~legitDomains['tld'].str.contains('\`|\.')]
dgaDomains = dgaDomains[~dgaDomains['tld'].str.contains('\`|\.')]

In [3]:
legitDoamins = legitDomains.drop_duplicates()
dgaDomains = dgaDomains.drop_duplicates()

legitDomains['label'] = 0
dgaDomains['label'] = 1

In [4]:
allDomains = concat([legitDomains, dgaDomains], ignore_index=True)
allDomains = allDomains.sample(frac=1).reset_index(drop=True)

X, y = allDomains['tld'], allDomains['label']

In [5]:
validChars = {x: idx + 1 for idx, x in enumerate(set(''.join(X))) }

maxFeatures = len(validChars) + 1
maxlen = np.max([len(x) for x in X])

In [6]:
X = [[validChars[y] for y in x] for x in X]
X = pad_sequences(X, maxlen=maxlen)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [7]:
model = Sequential()
model.add(Embedding(maxFeatures, 128, input_length=maxlen))
model.add(LSTM(128))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='rmsprop')

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.1, verbose=2)

2024-02-12 19:39:22.389434: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 179036408 exceeds 10% of free system memory.


Epoch 1/10


2024-02-12 19:58:09.862660: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19893072 exceeds 10% of free system memory.


11281/11281 - 1173s - loss: 0.0118 - val_loss: 0.0091 - 1173s/epoch - 104ms/step
Epoch 2/10


In [1]:
domain = [[validChars[ch] for ch in tldextract.extract('wikipedia.com').domain]]
domain = pad_sequences(domain, maxlen=maxlen)

model.predict(domain)

NameError: name 'tldextract' is not defined

In [19]:
probs = model.predict(X_test)

tn, fp, fn, tp = confusion_matrix(y_test, probs > 0.5).ravel()

print('TP: %d\nTN: %d\nFP: %d\nFN: %d\n' % (tp, tn, fp, fn))
print('FP rate: %.3f%%\nFN rate: %.3f##\n' % (fp / (fp + tn) * 100, fn (fn + tp) * 100))

print('Sensitivity: %.3f%%\nSpecificity: %.3f%%\nAccuracy: %.3f%%\n' (
    tp / (tp + fn),
    tn / (tn + fp),
    (tp + fn) / (tp + tn + fp + fn)
))

print('AUC: %.3f%%' % roc_auc_score(y_test, probs))

   3/6267 [..............................] - ETA: 2:41

<>:8: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
<>:8: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?


6267/6267 [==============================] - 171s 27ms/step
TP: 140
TN: 199943
FP: 51
FN: 400



TypeError: 'numpy.int64' object is not callable